In [5]:
from foundation_model.models import FlexibleMultiTaskModel
from foundation_model.models.model_config import RegressionTaskConfig
from foundation_model.data import CompoundDataModule, CompoundDataset

In [6]:
import lightning as pl
import torch

from lightning.pytorch.trainer import Trainer
from lightning.pytorch.callbacks import ModelCheckpoint, EarlyStopping
from lightning.pytorch.loggers.csv_logs import CSVLogger
from lightning.pytorch.loggers.tensorboard import TensorBoardLogger

In [14]:
task_configs =[
        RegressionTaskConfig(
            name="task1_xxxxxx",
            data_column="Task 1 (normalized)",
            dims=[64, 32, 1]),
    ]

model = FlexibleMultiTaskModel(
    shared_block_dims=[290, 128],
    task_configs=task_configs,
)

2025-06-27 09:53:51.623 | INFO     | flexible_multi_task_model:__init__:207 - Learnable task uncertainty (task_log_sigmas) is ENABLED.
2025-06-27 09:53:51.625 | INFO     | flexible_multi_task_model:__init__:227 - Initializing FlexibleMultiTaskModel...
2025-06-27 09:53:51.625 | INFO     | flexible_multi_task_model:__init__:228 - Registered Task Heads:
2025-06-27 09:53:51.626 | INFO     | flexible_multi_task_model:__init__:235 -           name       type  enabled
2025-06-27 09:53:51.626 | INFO     | flexible_multi_task_model:__init__:235 -   task1_xxxxxx REGRESSION     True
2025-06-27 09:53:51.627 | INFO     | flexible_multi_task_model:__init__:239 - FlexibleMultiTaskModel Structure:
2025-06-27 09:53:51.627 | INFO     | flexible_multi_task_model:__init__:243 -   FlexibleMultiTaskModel(
2025-06-27 09:53:51.627 | INFO     | flexible_multi_task_model:__init__:243 -     (task_log_sigmas): ParameterDict(  (task1_xxxxxx): Parameter containing: [torch.FloatTensor of size ])
2025-06-27 09:53:51.

In [ ]:
trainer = Trainer(
    max_epochs=10,
    accelerator="auto",
    devices=1,
    logger=[
        CSVLogger("logs", name="test_1"),
        TensorBoardLogger(
            save_dir="logs",
            name="tensorboard_logs",
            default_hp_metric=False,
        ),
    ],
    callbacks=[
        ModelCheckpoint(
            dirpath="checkpoints",
            filename="best-{epoch:02d}-{val_loss:.2f}",
            monitor="val_final_loss",
            mode="min",
            save_top_k=1,
        ),
        EarlyStopping(
            monitor="val_final_loss",
            patience=3,
            mode="min",
        ),
    ],
)

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


In [2]:
import pandas as pd

a = pd.read_parquet("/Users/liuchang/projects/foundation_model/data/qc_ac_te_mp_dos_reformat_20250615.pd.parquet")
a

,formula,Material type,Space group,composition,elements,Band gap,Density,Efermi,Final energy per atom,Formation energy per atom,...,Electrical resistivity (normalized),Power factor (normalized),Seebeck coefficient (normalized),Thermal conductivity (normalized),ZT (normalized),Magnetic susceptibility (normalized),DOS energy,DOS density,DOS density (normalized),split
id,,,,,,,,,,,,,,,,,,,,,
mp-111,Ne,others,Fm-3m,"{'A0+': None, 'Ac': None, 'Ag': None, 'Al': No...",[Ne],11.6898,1.815607,-9.772494,-1.912824,0.000000,...,None,None,None,None,None,None,None,None,None,val
mp-23158,He,others,Im-3m,"{'A0+': None, 'Ac': None, 'Ag': None, 'Al': No...",[He],17.6377,0.384324,-13.591344,-0.313901,0.000000,...,None,None,None,None,None,None,None,None,None,train
mp-23155,Ar,others,Fm-3m,"{'A0+': None, 'Ac': None, 'Ag': None, 'Al': No...",[Ar],8.5132,1.720053,-5.482198,-4.858515,0.000000,...,None,None,None,None,None,None,None,None,None,train
mp-1525632,F2,others,Cmce,"{'A0+': None, 'Ac': None, 'Ag': None, 'Al': No...",[F],2.8979,2.366529,-4.635707,-3.126028,0.000000,...,None,None,None,None,None,None,None,None,None,val
mp-1095223,OF2,others,Pnma,"{'A0+': None, 'Ac': None, 'Ag': None, 'Al': No...","[F, O]",2.1241,2.196288,-2.171794,-3.421278,-0.496056,...,None,None,None,None,None,None,None,None,None,val
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
starry-193435,Fe1.9Ni0.1VAL,others,None,"{'A0+': 1.0, 'Ac': None, 'Ag': None, 'Al': Non...","[Fe, Ni, V, A0+, L0+]",NaN,NaN,NaN,NaN,NaN,...,"[-0.30261529812521487, -0.3027711534321466, -0...",None,"[-0.1539958251721523, -0.15957512903072363, -0...",None,None,None,None,None,None,train
starry-193434,Fe1.95Ni0.05VAL,others,None,"{'A0+': 1.0, 'Ac': None, 'Ag': None, 'Al': Non...","[Fe, Ni, V, A0+, L0+]",NaN,NaN,NaN,NaN,NaN,...,"[-0.3016918542233598, -0.3018193776646024, -0....",None,"[-0.2053880854552989, -0.2212722739402773, -0....",None,None,None,None,None,None,train
starry-193433,Fe2VAL,others,None,"{'A0+': 1.0, 'Ac': None, 'Ag': None, 'Al': Non...","[Fe, V, A0+, L0+]",NaN,NaN,NaN,NaN,NaN,...,"[-0.2994169953908805, -0.2996572714931658, -0....",None,"[-0.04574929789083675, -0.027669073388469956, ...",None,None,None,None,None,None,train


In [ ]:
data_module = CompoundDataModule(
    task_configs=task_configs,
    formula_desc_source="data/formula_desc.csv",
    attributes_source="data/attributes.csv",
)

In [ ]:
trainer.fit(
    model=model,
    datamodule=data_module,
    ckpt_path=None,
)

In [ ]:
trainer.test(
    model=model,
    datamodule=data_module,
    ckpt_path="xxxxx/checkpoints/best-epoch=00-val_loss=0.00.ckpt",
)

In [ ]:
trainer.predict(
    model=model,
    datamodule=data_module,
    ckpt_path="xxxxx/checkpoints/best-epoch=00-val_loss=0.00.ckpt",
)